In [29]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer


In [2]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [3]:
data_path = '../data/csv/'
file_paths = ['links.csv', 'movies.csv', 'ratings.csv', 'tags.csv']

In [4]:
for file in file_paths:
    pass

In [5]:
links_df = pd.read_csv(os.path.join(data_path, file_paths[0]))
movies_df = pd.read_csv(os.path.join(data_path, file_paths[1]))
ratings_df = pd.read_csv(os.path.join(data_path, file_paths[2]))
tags_df = pd.read_csv(os.path.join(data_path, file_paths[3]))

In [6]:
links_df.shape, movies_df.shape, ratings_df.shape, tags_df.shape

((9742, 3), (9742, 3), (100836, 4), (3683, 4))

In [7]:
# links_df.head()

In [8]:
movies_df.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [9]:
ratings_df.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [10]:
tags_df.columns

Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')

In [11]:
tags_df['tag'].value_counts()

In Netflix queue       131
atmospheric             36
superhero               24
thought-provoking       24
Disney                  23
                      ... 
Tradition!               1
ROBOTS AND ANDROIDS      1
male nudity              1
long takes               1
unintelligent            1
Name: tag, Length: 1589, dtype: int64

In [12]:
ratings_df['userId'].value_counts()

414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
406      20
595      20
569      20
431      20
442      20
Name: userId, Length: 610, dtype: int64

In [13]:
len(tags_df['movieId'].unique()), len(movies_df['movieId'].unique())

(1572, 9742)

In [14]:
mr_df = pd.merge(ratings_df, movies_df,  
                  how='left', on=['movieId'])

In [15]:
mr_df['genres_list'] = np.nan
counter = 0
for i, r in enumerate(mr_df['genres']):
    genre = r.split('|')
    mr_df['genres_list'][i] = genre
    counter += 1
    if counter%1000==0:
        print(f'{counter/1000}%')

/Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/sam/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1.0%
2.0%
3.0%
4.0%
5.0%
6.0%
7.0%
8.0%
9.0%
10.0%
11.0%
12.0%
13.0%
14.0%
15.0%
16.0%
17.0%
18.0%
19.0%
20.0%
21.0%
22.0%
23.0%
24.0%
25.0%
26.0%
27.0%
28.0%
29.0%
30.0%
31.0%
32.0%
33.0%
34.0%
35.0%
36.0%
37.0%
38.0%
39.0%
40.0%
41.0%
42.0%
43.0%
44.0%
45.0%
46.0%
47.0%
48.0%
49.0%
50.0%
51.0%
52.0%
53.0%
54.0%
55.0%
56.0%
57.0%
58.0%
59.0%
60.0%
61.0%
62.0%
63.0%
64.0%
65.0%
66.0%
67.0%
68.0%
69.0%
70.0%
71.0%
72.0%
73.0%
74.0%
75.0%
76.0%
77.0%
78.0%
79.0%
80.0%
81.0%
82.0%
83.0%
84.0%
85.0%
86.0%
87.0%
88.0%
89.0%
90.0%
91.0%
92.0%
93.0%
94.0%
95.0%
96.0%
97.0%
98.0%
99.0%
100.0%


In [16]:
mr_df.isna().sum()

userId         0
movieId        0
rating         0
timestamp      0
title          0
genres         0
genres_list    0
dtype: int64

In [17]:
mr_df['userId']

0           1
1           1
2           1
3           1
4           1
         ... 
100831    610
100832    610
100833    610
100834    610
100835    610
Name: userId, Length: 100836, dtype: int64

In [18]:
user_encode = LabelEncoder()
mr_df['user'] = user_encode.fit_transform(mr_df['userId'].values)
n_users = mr_df['user'].nunique()

movie_encode = LabelEncoder()
mr_df['movie'] = user_encode.fit_transform(mr_df['movieId'].values)
n_movies = mr_df['movie'].nunique()

mr_df['rating'] = mr_df['rating'].values.astype(np.float32)
min_rating = min(mr_df['rating'])
max_rating = max(mr_df['rating'])
'n_users', n_users, 'n_movies', n_movies, 'min_rating', min_rating, 'max_rating', max_rating


('n_users', 610, 'n_movies', 9724, 'min_rating', 0.5, 'max_rating', 5.0)

In [19]:
X = mr_df[['user', 'movie']].values
y = mr_df['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((90752, 2), (10084, 2), (90752,), (10084,))

In [20]:
n_factors = 50
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [27]:
import pyspark.sql
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml.feature import VectorAssembler

In [22]:
spark = SparkSession(SparkContext())

In [23]:
list(mr_df.columns)

['userId',
 'movieId',
 'rating',
 'timestamp',
 'title',
 'genres',
 'genres_list',
 'user',
 'movie']

In [28]:
mr_df['genres_list']

0         [Adventure, Animation, Children, Comedy, Fantasy]
1                                         [Comedy, Romance]
2                                 [Action, Crime, Thriller]
3                                       [Mystery, Thriller]
4                                [Crime, Mystery, Thriller]
                                ...                        
100831                            [Drama, Horror, Thriller]
100832                            [Action, Crime, Thriller]
100833                                             [Horror]
100834                                     [Action, Sci-Fi]
100835                     [Action, Crime, Drama, Thriller]
Name: genres_list, Length: 100836, dtype: object

In [39]:

mlb = MultiLabelBinarizer()

encoded_genres = pd.DataFrame(mlb.fit_transform(mr_df['genres_list']),columns=mlb.classes_, index=mr_df.index)

In [41]:
mr_df = pd.concat([mr_df,encoded_genres],axis=1)

In [48]:
spark_df = spark.createDataFrame(mr_df.drop(['timestamp', 'title', 'genres', 'genres_list'], axis=1))

In [50]:
assembler = VectorAssembler(
inputCols=list(spark_df.columns),
outputCol='features')
trainingData = assembler.transform(spark_df)
# trainingData.show()

In [52]:
als = ALS(rank=10, maxIter=5, seed=0)

In [56]:
als.setItemCol("userId")
als.fit(trainingData)

ALS_973c98569f26

In [57]:
als.rank

Param(parent='ALS_973c98569f26', name='rank', doc='rank of the factorization')